In [1]:
:opt no-lint

# 파서의 타입
```haskell
type Parser = String -> Tree                -- 문자열을 처리해 Tree로 변환
type Parser = String -> (Tree, String)    -- 처리하고 남은 뒷부분 문자열 고려
type Parser = String -> [(Tree, String)]  -- 실패할 경우 길이 0인 빈 리스트
type Parser a = String -> [(a, String)]   -- 특정 Tree가 대신 타입 a로 일반화
type Parser a = [Char] -> [(a, [Char])]   -- 윗줄과 같음  String = [Char]
type Parser tok a = [tok] -> [(a, [tok])] -- Char 대신 토큰 타입 tok로 일반화
```

In [2]:
type Parser tok a = [tok] -> [(a, [tok])]

In [3]:
return :: a -> Parser tok a -- 입력 토큰열 ts를 소비하지 않고
return v = \ts -> [(v,ts)]  -- 그냥 v를 리턴하며 성공하는 파서

failure :: Parser tok a -- 입력에 관계없이 무조건 실패하는 파서
failure = \_ -> []

In [4]:
(return 1) "abc"
failure    "abc"

[(1,"abc")]

[]

In [5]:
item :: Parser tok tok
item []     = []       -- 길이 0인 토큰열에 대해서는 실패
item (t:ts) = [(t,ts)] -- 맨 앞의 토큰 t하나만 처리해 t를 리턴 

In [6]:
item ""
item "abc"

[]

[('a',"bc")]

In [7]:
(>>=) :: Parser tok a -> (a -> Parser tok b) -> Parser tok b
p1 >>= pf = \s -> [ (v2,s2) | (v1,s1) <- p1 s, -- 이어붙이기 sequencing
                              (v2,s2) <- (pf v1) s1 ]

In [8]:
( item >>= \_  ->  item                            ) "abcd"
( item >>= \c1 ->  item >>= \c2 ->  return [c1,c2] ) "abcd" 

[('b',"cd")]

[("ab","cd")]

In [9]:
( item >>= \c -> if c=='h' then item else failure ) "hello"
( item >>= \c -> if c=='h' then item else failure ) "world"

[('e',"llo")]

[]

In [10]:
( item >>=                                \c1 ->
  (if c1=='h' then item else failure) >>= \c2 ->
  return [c1,c2]                                 ) "hello"
( item >>=                                \c1 ->
  (if c1=='h' then item else failure) >>= \c2 ->
  return [c1,c2]                                 ) "world"

[("he","llo")]

[]

In [11]:
(<|>) :: Parser tok a -> Parser tok a -> Parser tok a  -- 선택 choice
p1 <|> p2 = \s -> case p1 s of
                    []  -> p2 s  -- 첫번째 파서가 실패하면 두번째로
                    vs1 -> vs1   -- 첫번째가 성공하면 첫번째만으로

In [12]:
item                  "ab"
(item <|> return '?') "ab"

[('a',"b")]

[('a',"b")]

In [13]:
item                  ""
(item <|> return '?') ""

[]

[('?',"")]

In [14]:
sat :: (tok -> Bool) -> Parser tok tok
sat test = item >>= \t ->          -- 토큰 하나를 읽어들여
           if test t then return t -- 조건에 맞는 경우에만 성공하고 
                     else failure  -- 그렇지 않으면 실패하는 파서

In [15]:
( sat (=='h') >>= \c1 -> item >>= \c2 -> return [c1,c2] ) "hello"
( sat (=='h') >>= \c1 -> item >>= \c2 -> return [c1,c2] ) "world"

[("he","llo")]

[]

In [16]:
many  :: Parser tok a -> Parser tok [a]
many  p = many1 p <|> return []

many1 :: Parser tok a -> Parser tok [a]
many1 p = p      >>= \v  ->
          many p >>= \vs ->
          return (v:vs)

In [17]:
( many (sat (=='h')) ) "hhhi"
( many (sat (=='h')) ) "iiii"

[("hhh","i")]

[("","iiii")]

In [18]:
( many1 (sat (=='h')) ) "hhhi"
( many1 (sat (=='h')) ) "iiii"

[("hhh","i")]

[]

# 문자열을 처리하는 파서

In [19]:
import Data.Char ( isDigit, isLower, isUpper,
                   isAlpha, isAlphaNum, isSpace )
digit = sat isDigit
lower = sat isLower
upper = sat isUpper
letter = sat isAlpha
alphanum = sat isAlphaNum
space = sat isSpace

In [20]:
:type isDigit
:type digit

isDigit :: Char -> Bool

digit :: Parser Char Char

In [21]:
digit "123"
digit "a23"

[('1',"23")]

[]

In [22]:
char :: Char -> Parser Char Char    -- char c는 주어진 글자 c와 첫글자가
char c = sat (==c)                  -- 일치하는 경우에만 성공하는 파서

string :: String -> Parser Char String  -- string s는 주어진 문자열 s와
string []     = return []               -- 앞부분이 일치하는 경우에만 성공
string (c:cs) = char c    >>= \_ ->
                string cs >>= \_ ->
                return (c:cs)

In [23]:
char 'h' "hello"
char 'h' "world"

[('h',"ello")]

[]

In [24]:
string ("abc") "abcdef"
string ("abc") "ab1234"

[("abc","def")]

[]

In [25]:
nat :: Parser Char Int
nat = many1 digit >>= \s ->
      return (read s)

ident :: Parser Char String
ident = lower          >>= \c  ->
        many1 alphanum >>= \cs ->
        return (c:cs)

spaces, spaces1 :: Parser Char ()
spaces  = many  space >>= \_ -> return ()
spaces1 = many1 space >>= \_ -> return ()

In [26]:
(many1 digit) "123def"
nat           "123def"

[("123","def")]

[(123,"def")]

In [27]:
ident "abc123defghi"
ident "abc123d  ghi"
ident "123abcd  ghi"

[("abc123defghi","")]

[("abc123d","  ghi")]

[]

In [28]:
spaces "  abc"
spaces "abc"

[((),"abc")]

[((),"abc")]

In [29]:
spaces1 "  abc"
spaces1 "abc"

[((),"abc")]

[]

# 토큰화

In [30]:
data Tok = KW String -- 키워드
         | ID String -- 변수 이름
         | INT Int   -- 정수
         | LP        -- (
         | RP        -- )
         | LAM       -- \
         | DOT       -- .
         | ADD       -- +
         deriving (Eq,Ord,Show)

In [31]:
word :: Parser Char Tok
word = ident  >>= \s ->
       if s `elem` ["if","then","else"] then return (KW s)
                                        else return (ID s)

natural :: Parser Char Tok
natural = nat >>= \n -> return (INT n)

In [32]:
"if" `elem` ["if","then","else"]
"hi" `elem` ["if","then","else"]

True

False

In [33]:
word "if then else  "
word "ifthen  else  "

[(KW "if"," then else  ")]

[(ID "ifthen","  else  ")]

In [34]:
(many word) "if then else  "
(many word) "ifthen  else  "

[([KW "if"]," then else  ")]

[([ID "ifthen"],"  else  ")]

In [35]:
tok :: Parser Char a -> Parser Char a
tok p = p      >>= \v ->
        spaces >>= \_ ->
        return v

In [36]:
(many (tok word)) "if then else  "
(many (tok word)) "ifthen  else  "

[([KW "if",KW "then",KW "else"],"")]

[([ID "ifthen",KW "else"],"")]

In [37]:
(many (tok word <|> tok natural)) "if b1 then 123 else x3  "
(many (tok word <|> tok natural)) "if b1 then 123 else 3 + (\\y. y) "

[([KW "if",ID "b1",KW "then",INT 123,KW "else",ID "x3"],"")]

[([KW "if",ID "b1",KW "then",INT 123,KW "else",INT 3],"+ (\\y. y) ")]

# 문법분석

In [38]:
data Expr = Lit Int            -- n
          | Add Expr Expr      -- e1 + e2
          | If Expr Expr Expr  -- if e then e1 else e0
          deriving (Eq, Ord, Show)

In [39]:
aexp, aexp0, aexp1 :: Parser Tok Expr
aexp = aexp0  -- 덧셈식만 처리하는 파서

-- EBNF로는 aexp0 ::= { aexp1 "+" } aexp1
aexp0 = many ( aexp1       >>= \e ->
               sat (==ADD) >>= \_ -> 
               return e              )  >>= \es ->
        aexp1                           >>= \e' ->
        return (foldl1 Add (es ++ [e']))  -- 덧셈을 좌결합으로 처리

-- EBNF로는 aexp1 ::= lit | "(" aexp0 ")"
aexp1 = lit <|> paren aexp0

lit = sat isINT  >>= \(INT n) -> return (Lit n)
    where isINT (INT _) = True
          isINT _       = False

paren p = sat (==LP) >>= \_ ->
          p          >>= \e ->
          sat (==RP) >>= \_ ->
          return e

In [40]:
lit [INT 1]             -- 1
lit [INT 1, ADD, INT 2] -- 1 + 2

[(Lit 1,[])]

[(Lit 1,[ADD,INT 2])]

In [41]:
aexp1 [INT 2]                     -- 2
aexp1 [INT 2, ADD, INT 3]         -- 2 + 3
aexp1 [LP, INT 2, ADD, INT 3, RP] -- (2 + 3)

[(Lit 2,[])]

[(Lit 2,[ADD,INT 3])]

[(Add (Lit 2) (Lit 3),[])]

In [42]:
aexp [INT 1, ADD, INT 2]                     -- 1 + 2
aexp [INT 1, ADD, LP, INT 2, ADD, INT 3, RP] -- 1 + (2 + 3)

[(Add (Lit 1) (Lit 2),[])]

[(Add (Lit 1) (Add (Lit 2) (Lit 3)),[])]

In [43]:
expr, expr0, expr1 :: Parser Tok Expr
expr = expr0 -- Expr에 대한 파서

-- EBNF로는 expr0 ::= { expr1 "+" } (expr1 | cexpr)
expr0 = many ( expr1       >>= \e ->
               sat (==ADD) >>= \_ -> 
               return e              )  >>= \es ->
        (expr1 <|> cexpr)               >>= \e' ->
        return (foldl1 Add (es ++ [e']))

expr1 = lit <|> paren expr0

cexpr = sat (== KW "if")   >>= \_  -> expr0 >>= \e  ->
        sat (== KW "then") >>= \_  -> expr0 >>= \e1 ->
        sat (== KW "else") >>= \_  -> expr0 >>= \e0 ->
        return (If e e1 e0)

In [46]:
expr [INT 1,
      ADD,
      KW "if", INT 10,KW "then", INT 20, KW "else", INT 30]
expr [KW "if", INT 10,
      KW "then", INT 20,
      KW "else", INT 30, ADD, INT 1]
expr [LP, KW "if", INT 10,KW "then", INT 20, KW "else", INT 30, RP,
      ADD,
      INT 1]

[(Add (Lit 1) (If (Lit 10) (Lit 20) (Lit 30)),[])]

[(If (Lit 10) (Lit 20) (Add (Lit 30) (Lit 1)),[])]

[(Add (If (Lit 10) (Lit 20) (Lit 30)) (Lit 1),[])]

\section*{연습문제}
1. `[25]`번 셀의 `nat :: Parser Char Int`는
   `"12"`나 `"0"`처럼 자연수(0 이상의 정수)를 나타내는 문자열은 처리할 수 있지만
   `"-123"`처럼 음의 정수를 나타내는 문자열을 처리하지 못한다.
   `"12"`나 `"0"`처럼 자연수를 나타내는 문자열과 `"-123"`처럼 음의 정수를 나타내는
   문자열을 모두 처리할 수 있는, 즉 십진수 정수를 나타내는 문자열을 처리하는
   `int :: Parser Char Int`를 작성해 보라.
1. 다음 두 파서가 어떤 문자열에 대해 처리 결과가 달라지는지 실행 사례를 들고
   어째서 그러한 차이점이 발생하는지도 설명해 보라.
   - `(string "if" <|> string "then" <|> string "else" <|> ident)`
   - `(ident <|> string "if" <|> string "then" <|> string "else")`
1. `[30]`번 셀의 `Tok`는 FAC언어(\ref{chap:FunArithEval}장)의 어휘에 대한
   토큰을 나타내는 데이터 타입이다. 맨 앞에도 공백문자들이 나타나는 것을 허용하면서
   FAC언어를 나타내는 문자열을 토큰화하는
   `lexFAC :: Parser Char Tok` 함수를 작성해 보라.
1. TODO
1. TODO